In [47]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from FileCounter import fileCount
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

In [48]:
trainData = 'asl-alphabet/asl_alphabet_train/asl_alphabet_train'
testData = 'asl-alphabet/asl_alphabet_test/asl_alphabet_test'

In [49]:
train_transforms =  transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(trainData, transform=train_transforms)
test_dataset = datasets.ImageFolder(testData, transform=test_transforms)

In [50]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)


In [51]:

# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
#         self.fc1 = nn.Linear(64 * 37 * 37, 128)
#         self.fc2 = nn.Linear(128, 29)

#     def forward(self, x):
#         x = self.pool(nn.functional.relu(self.conv1(x)))
#         x = self.pool(nn.functional.relu(self.conv2(x)))
#         x = x.view(-1, 64 * 37 * 37)
#         x = nn.functional.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 25 * 25, 128)
        self.fc2 = nn.Linear(128, 29)

    def forward(self, x):
        x = self.pool1(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 25 * 25)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [52]:
net = CNN()

# Step 4: Train the model

# Move model and data to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

# Step 4: Train the model
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # move data to GPU

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 3.201
[1,   400] loss: 2.534
[1,   600] loss: 2.078
[1,   800] loss: 1.737
[1,  1000] loss: 1.476
[2,   200] loss: 1.127
[2,   400] loss: 0.969
[2,   600] loss: 0.861
[2,   800] loss: 0.782
[2,  1000] loss: 0.684
[3,   200] loss: 0.547
[3,   400] loss: 0.523
[3,   600] loss: 0.454
[3,   800] loss: 0.444
[3,  1000] loss: 0.401
[4,   200] loss: 0.340
[4,   400] loss: 0.322
[4,   600] loss: 0.289
[4,   800] loss: 0.270
[4,  1000] loss: 0.256
[5,   200] loss: 0.220
[5,   400] loss: 0.208
[5,   600] loss: 0.204
[5,   800] loss: 0.188
[5,  1000] loss: 0.188
[6,   200] loss: 0.145
[6,   400] loss: 0.161
[6,   600] loss: 0.158
[6,   800] loss: 0.150
[6,  1000] loss: 0.158
[7,   200] loss: 0.116
[7,   400] loss: 0.127
[7,   600] loss: 0.106
[7,   800] loss: 0.103
[7,  1000] loss: 0.096
[8,   200] loss: 0.091
[8,   400] loss: 0.077
[8,   600] loss: 0.089
[8,   800] loss: 0.105
[8,  1000] loss: 0.110
[9,   200] loss: 0.074
[9,   400] loss: 0.062
[9,   600] loss: 0.062
[9,   800] 

In [55]:
# Step 5: Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)  # move data to GPU
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 70 %
